**This notebook includes:**

**#01** Importing libraries

**#02** Exploring data

**#03** Wrangling data

**#04** Cleaning data

**#05** Getting descriptive statistics

**#06** Exporting data

**#01 Importing libraries**

In [5]:
import pandas as pd
import numpy as np
import os

In [7]:
#Importing data from csv
path = '/Users/gingermoore/Documents/06-2025 Airbnb Dallas'

In [9]:
df = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'calendar.csv'))

**#02 Exploring data**

In [11]:
df.head()

listing_id        date available   price  adjusted_price  minimum_nights  \
0       61878  2025-04-17         f  $75.00             NaN            30.0   
1       61878  2025-04-18         f  $75.00             NaN            30.0   
2       61878  2025-04-19         f  $75.00             NaN            30.0   
3       61878  2025-04-20         f  $75.00             NaN            30.0   
4       61878  2025-04-21         f  $75.00             NaN            30.0   

   maximum_nights  
0          1125.0  
1          1125.0  
2          1125.0  
3          1125.0  
4          1125.0

In [13]:
#Checking how many unique listing ids are in my dataset
df['listing_id'].nunique()

5349

In [100]:
df.shape

(1952367, 7)

In [15]:
#Checking data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1952367 entries, 0 to 1952366
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   listing_id      int64  
 1   date            object 
 2   available       object 
 3   price           object 
 4   adjusted_price  float64
 5   minimum_nights  float64
 6   maximum_nights  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 104.3+ MB


**#03 Wrangling data**

I'm going to change the data types so I can generate descriptive stats later on.

In [104]:
df['listing_id'] = df['listing_id'].astype(str)

In [105]:
df['date'] = pd.to_datetime(df['date'])

In [106]:
df['available_bool'] = df['available'].map({'t': True, 'f': False})

In [107]:
df['price'] = df['price'].str.replace(r'[\$,]', '', regex=True)

In [108]:
df['price'] = df['price'].astype(float)

**#04 Cleaning data**

In [110]:
#Checking for mixed-type data
for col in df.columns.tolist():
    weird = (df[[col]].map(type) !=df[[col]].
iloc[0].apply(type)).any(axis = 1)
    if len (df[weird]) > 0:
        print (col)

No mixed data types!

In [112]:
#Checking for missing values
df.isnull().sum()

listing_id              0
date                    0
available               0
price                   0
adjusted_price    1952367
minimum_nights          2
maximum_nights          2
available_bool          0
dtype: int64

In [113]:
df_null = df[df['minimum_nights'].isnull() == True]

In [114]:
df_null

listing_id       date available  price  adjusted_price  \
1130749  1106734717911768112 2025-04-16         t  165.0             NaN   
1236235  1157962329129764054 2025-04-16         f  171.0             NaN   

         minimum_nights  maximum_nights  available_bool  
1130749             NaN             NaN            True  
1236235             NaN             NaN           False

It appears that the adjusted_price column is all null for some reason. I do not believe it will be necessary for my analysis, so I will drop that column. I will also drop the two observations above that have null values for minimum and maximum nights.

In [116]:
#Checking for duplicates
df_dups = df[df.duplicated()]

In [117]:
df_dups

Empty DataFrame
Columns: [listing_id, date, available, price, adjusted_price, minimum_nights, maximum_nights, available_bool]
Index: []

There are no duplicates in this dataset.

In [21]:
df_clean = df.drop(columns = ['adjusted_price'])

In [23]:
df_clean

listing_id        date available   price  minimum_nights  \
0             61878  2025-04-17         f  $75.00            30.0   
1             61878  2025-04-18         f  $75.00            30.0   
2             61878  2025-04-19         f  $75.00            30.0   
3             61878  2025-04-20         f  $75.00            30.0   
4             61878  2025-04-21         f  $75.00            30.0   
...             ...         ...       ...     ...             ...   
1952362    36351143  2026-04-12         f  $82.00             5.0   
1952363    36351143  2026-04-13         f  $82.00             5.0   
1952364    36351143  2026-04-14         f  $82.00             5.0   
1952365    36351143  2026-04-15         f  $82.00             5.0   
1952366    36351143  2026-04-16         f  $82.00             5.0   

         maximum_nights  
0                1125.0  
1                1125.0  
2                1125.0  
3                1125.0  
4                1125.0  
...                 ...  
1952362          1125.0  
1952363          1125.0  
1952364          1125.0  
1952365          1125.0  
1952366          1125.0  

[1952367 rows x 6 columns]

In [25]:
#Dropping null values from previous check
df_clean_2 = df_clean.drop([1130749, 1236235])

In [27]:
df_clean_2.isnull().sum()

listing_id        0
date              0
available         0
price             0
minimum_nights    0
maximum_nights    0
dtype: int64

In [146]:
#Checking to make sure the rows were dropped
df_clean_2.shape

(1952365, 7)

**#05 Getting descriptive statistics**

In [149]:
df_clean_2.describe()

date         price  minimum_nights  \
count                        1952365  1.952365e+06    1.952365e+06   
mean   2025-10-15 22:11:36.738981632  3.662788e+02    9.078510e+00   
min              2025-04-16 00:00:00  1.000000e+01    1.000000e+00   
25%              2025-07-17 00:00:00  9.000000e+01    1.000000e+00   
50%              2025-10-16 00:00:00  1.370000e+02    2.000000e+00   
75%              2026-01-15 00:00:00  2.600000e+02    7.000000e+00   
max              2026-04-27 00:00:00  1.500000e+04    3.650000e+02   
std                              NaN  1.168279e+03    1.805789e+01   

       maximum_nights  
count    1.952365e+06  
mean     2.409540e+06  
min      1.000000e+00  
25%      3.650000e+02  
50%      3.650000e+02  
75%      1.125000e+03  
max      2.147484e+09  
std      7.188325e+07

**#06 Exporting data**

In [152]:
path = '/Users/gingermoore/Documents/06-2025 Airbnb Dallas/02 Data/Transformed Data'

In [154]:
df_clean_2.to_pickle(os.path.join(path, 'calendar_wrangled.pkl'))